In [18]:
import os
access_token = os.environ["ACCESS_TOKEN"]
account_id_demo = os.environ["ACCOUNT_DEMO"]
account_id_hedge = os.environ["ACCOUNT_HEDGE"]
import oandapyV20
import oandapyV20.endpoints.transactions as trans
import pandas as pd

In [32]:
client = oandapyV20.API(access_token=access_token)
param = { "id": 900 }
r = trans.TransactionsSinceID(accountID=account_id_demo,  params=param)
trades= client.request(r)
param1 = { "id": 0 }
request_hedge = trans.TransactionsSinceID(accountID=account_id_hedge,  params=param1)
trades_hedge= client.request(request_hedge)


In [35]:
all_transactions_hedge = pd.DataFrame(trades_hedge['transactions'])
all_transactions = pd.DataFrame(trades['transactions'])
all_transactions = pd.concat([all_transactions_hedge,all_transactions])
all_transactions['time'] = pd.to_datetime(all_transactions['time'])
all_transactions['date'] = all_transactions['time'].dt.date
all_transactions = all_transactions[all_transactions['date'] == all_transactions['date'].max()]
all_transactions['pl'] = pd.to_numeric(all_transactions['pl'])
print(all_transactions[all_transactions['reason'].isin(['TRAILING_STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby(['accountID','reason'])['pl'].sum())
print(all_transactions['pl'].sum())

accountID             reason                  
101-004-31059296-001  TAKE_PROFIT_ORDER           2353.7937
                      TRAILING_STOP_LOSS_ORDER   -5911.3750
101-004-31059296-003  TAKE_PROFIT_ORDER           3757.7192
                      TRAILING_STOP_LOSS_ORDER   -4347.5184
Name: pl, dtype: float64
-2733.718300000001


In [27]:
pd.to_numeric(all_transactions['halfSpreadCost']).sum()

4208.531800000001

In [5]:
orders = pd.read_csv('')
orders['Time'] = pd.to_datetime(orders['Time'])

In [6]:
all_transactions['id'] = pd.to_numeric(all_transactions['id'])
all_transactions[all_transactions['id'].isin(orders['Order_ID'])].head(2)

,id,accountID,userID,batchID,requestID,time,type,divisionID,siteID,accountUserID,...,rejectReason,gtdTime,guaranteedStopLossOnFill,guaranteedExecutionPremium,longPositionCloseout,positionFinancings,baseHomeConversionCost,tradeReduced,distance,trailingStopLossOnFill
151,152,101-004-31059296-001,31059296,152,115394447449742676,2025-03-19 09:27:01.537154565+00:00,MARKET_ORDER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,154,101-004-31059296-001,31059296,154,61351307288090414,2025-03-19 13:07:01.312072734+00:00,MARKET_ORDER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
all_transactions['pl'] = pd.to_numeric(all_transactions['pl'])
all_transactions[all_transactions['reason'].isin(['TRAILING_STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby('reason')['pl'].sum()

reason
TAKE_PROFIT_ORDER           3577.3055
TRAILING_STOP_LOSS_ORDER   -4326.9194
Name: pl, dtype: float64

In [41]:
all_transactions[['id','type','reason','batchID','orderID','tradeID','pl','time']].head(2)

,id,type,reason,batchID,orderID,tradeID,pl,time
0,1,CREATE,NaN,1,NaN,NaN,NaN,2025-04-04 11:27:44.777790339+00:00
1,2,CLIENT_CONFIGURE,NaN,1,NaN,NaN,NaN,2025-04-04 11:27:44.777790339+00:00


In [11]:
trans_orders = _dntk.execute_sql(
  'WITH trading_orders AS (\n    SELECT \n        CAST(all1.batchID AS INTEGER) as batchID,\n        base.pl,\n        all1.time,\n        date_trunc(\'minute\', all1.time - INTERVAL \'7 minutes\') AS order_time\n    FROM all_transactions base \n    LEFT JOIN all_transactions order_filled \n        ON CAST(base.orderID AS INTEGER) = order_filled.id\n    LEFT JOIN all_transactions all1 \n        ON order_filled.batchID = all1.id\n    WHERE base.reason IN (\'TAKE_PROFIT_ORDER\', \'STOP_LOSS_ORDER\')\n)\nSELECT * FROM trading_orders\nORDER BY batchID ',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
trans_orders

,batchID,pl,time,order_time
0,4,-2.0423,2025-02-12 15:55:23.415344+00:00,2025-02-12 15:48:00+00:00
1,18,-2.7433,2025-02-13 02:15:20.220511+00:00,2025-02-13 02:08:00+00:00
2,22,-0.0079,2025-02-27 01:19:36.388815+00:00,2025-02-27 01:12:00+00:00
3,26,-0.0167,2025-02-27 01:25:41.603451+00:00,2025-02-27 01:18:00+00:00
4,38,-0.0159,2025-02-27 12:47:18.701539+00:00,2025-02-27 12:40:00+00:00
...,...,...,...,...
98,801,0.0006,2025-04-02 22:27:27.901929+00:00,2025-04-02 22:20:00+00:00
99,832,0.0003,2025-04-02 22:42:22.780999+00:00,2025-04-02 22:35:00+00:00
100,841,0.0001,2025-04-02 22:45:55.021637+00:00,2025-04-02 22:38:00+00:00
101,919,1292.8004,2025-04-03 18:56:01.961608+00:00,2025-04-03 18:49:00+00:00


In [14]:
all_transactions['date'] = all_transactions['time'].dt.date
all_transactions = all_transactions[all_transactions['date'] == all_transactions['date'].max()]
all_transactions['pl'] = pd.to_numeric(all_transactions['pl'])
all_transactions[all_transactions['reason'].isin(['TRAILING_STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby('reason')['pl'].sum()

reason
TAKE_PROFIT_ORDER           3243.2733
TRAILING_STOP_LOSS_ORDER   -2425.8276
Name: pl, dtype: float64

In [18]:
print(all_transactions[all_transactions['reason'].isin(['TRAILING_STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby('reason')['pl'].sum())
print(all_transactions[all_transactions['reason'].isin(['TRAILING_STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby('reason')['pl'].count())

reason
TAKE_PROFIT_ORDER           3243.2733
TRAILING_STOP_LOSS_ORDER   -2425.8276
Name: pl, dtype: float64
reason
TAKE_PROFIT_ORDER           3
TRAILING_STOP_LOSS_ORDER    9
Name: pl, dtype: int64


In [124]:
total_pl=all_transactions[all_transactions['reason'].isin(['TRAILING_STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby('reason')['pl'].sum()

In [126]:
total_pl['TRAILING_STOP_LOSS_ORDER'] + total_pl['TAKE_PROFIT_ORDER']

-358.82079999999996

In [128]:
pd.to_numeric(all_transactions['halfSpreadCost']).sum()

1284.5762

In [138]:
pd.to_numeric(all_transactions['baseFinancing']).sum()

0.0

In [134]:
all_transactions[all_transactions['id']=='987']

,id,accountID,userID,batchID,requestID,time,type,divisionID,siteID,accountUserID,...,guaranteedExecutionFeeHomeConversionCost,homeConversionCost,tradeID,triggerMode,tradesClosed,closedTradeID,tradeCloseTransactionID,distance,rejectReason,date
86,987,101-004-31059296-001,31059296,987,NaN,2025-04-07 03:53:47.489811909+00:00,ORDER_FILL,NaN,NaN,NaN,...,0.00000000000000,-4.11094908010440,NaN,NaN,"[{'tradeID': '980', 'units': '-1000000', 'real...",NaN,NaN,NaN,NaN,2025-04-07


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a127aa9e-0a77-4af9-a6ce-85e7a9b74042' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>